In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense
from sklearn.metrics import r2_score

# Load the new training data
new_data = pd.read_excel("25-12.xlsx")
new_data = new_data.drop(["id", "temp", "w_status"], axis=1)
new_data['date_v'] = pd.to_datetime(new_data['date_v'])
new_data['unix_time'] = new_data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))
new_data = new_data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()
new_data['yukh_scaled'] = scaler_yukh.fit_transform(new_data[['yukh']])
new_data['reg_scaled'] = scaler_reg.fit_transform(new_data[['reg']])
new_data['power_scaled'] = scaler_power.fit_transform(new_data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
new_data = new_data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = new_data['yukh_scaled'][i:i+window_size].values
        window_reg = new_data['reg_scaled'][i:i+window_size].values
        window_power = new_data['power_scaled'][i:i+window_size].values
        target = new_data['yukh_scaled'][i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme
window_size = 64
# Create time series data for the new training set
X_new, y_new = create_time_series_data(new_data, window_size)

# Split the new data into training and testing sets
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X_new, y_new, test_size=0.20, random_state=42)

# Load the pre-trained model
model = load_model('model9481.h5')

# Retrain the model with the new data
model.fit(X_train_new, y_train_new, epochs=100, batch_size=16, validation_data=(X_test_new, y_test_new))

# Save the retrained model
model.save('retrained_model.h5')

# Test the retrained model on a separate test set if needed
test_predictions_new = model.predict(X_test_new)
r2_test_new = r2_score(y_test_new, test_predictions_new)

print(f'Retrained Model Test R-squared score: {r2_test_new}')


Epoch 1/100
2/2 [==============================] - 3s 604ms/step - loss: 0.0023 - val_loss: 0.0020
Epoch 2/100
2/2 [==============================] - 0s 110ms/step - loss: 0.0024 - val_loss: 0.0029
Epoch 3/100
2/2 [==============================] - 0s 114ms/step - loss: 0.0028 - val_loss: 0.0031
Epoch 4/100
2/2 [==============================] - 0s 111ms/step - loss: 0.0024 - val_loss: 0.0028
Epoch 5/100
2/2 [==============================] - 0s 112ms/step - loss: 0.0035 - val_loss: 0.0023
Epoch 6/100
2/2 [==============================] - 0s 112ms/step - loss: 0.0025 - val_loss: 0.0017
Epoch 7/100
2/2 [==============================] - 0s 125ms/step - loss: 0.0036 - val_loss: 0.0020
Epoch 8/100
2/2 [==============================] - 0s 112ms/step - loss: 0.0039 - val_loss: 0.0027
Epoch 9/100
2/2 [==============================] - 0s 111ms/step - loss: 0.0030 - val_loss: 0.0035
Epoch 10/100
2/2 [==============================] - 0s 113ms/step - loss: 0.0026 - val_loss: 0.0033
Epoch 11/

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 418ms/step
Retrained Model Test R-squared score: 0.9282133495004986


In [18]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# Verileri yükle
data = pd.read_excel("train.xlsx")
data = data.drop(["id", "temp", "w_status"], axis=1)

# 'date' sütununu datetime formatına çevirme
data['date_v'] = pd.to_datetime(data['date_v'])

# 'date' sütununu Unix zaman damgasına çevirme
data['unix_time'] = data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))

# Gereksiz sütunu ve orijinal tarih sütununu kaldırma
data = data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()

data['yukh_scaled'] = scaler_yukh.fit_transform(data[['yukh']])
data['reg_scaled'] = scaler_reg.fit_transform(data[['reg']])
data['power_scaled'] = scaler_power.fit_transform(data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
data = data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = data['yukh_scaled'][i:i+window_size].values
        window_reg = data['reg_scaled'][i:i+window_size].values
        window_power = data['power_scaled'][i:i+window_size].values
        target = data['yukh_scaled'][i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme
window_size = 64

# Giriş ve hedef verilerini oluşturma
X, y = create_time_series_data(data, window_size)

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

# Yüklü modeli yükleme
loaded_model = load_model('retrained_model.h5')

# Modeli tekrar eğitme
loaded_model.fit(X_train, y_train, epochs=50, batch_size=16, validation_data=(X_test, y_test))

# Test seti üzerinde tahmin yapma
test_predictions = loaded_model.predict(X_test)
r2_test = r2_score(y_test, test_predictions)

print(f'Modelin Test R-kare skoru: {r2_test}')


Epoch 1/50
144/144 [==============================] - 15s 92ms/step - loss: 7.5118e-04 - val_loss: 2.7025e-04
Epoch 2/50
144/144 [==============================] - 13s 93ms/step - loss: 5.0988e-04 - val_loss: 2.6260e-04
Epoch 3/50
144/144 [==============================] - 13s 87ms/step - loss: 4.7885e-04 - val_loss: 2.6899e-04
Epoch 4/50
144/144 [==============================] - 12s 87ms/step - loss: 4.2963e-04 - val_loss: 2.6933e-04
Epoch 5/50
144/144 [==============================] - 13s 87ms/step - loss: 4.3326e-04 - val_loss: 2.8026e-04
Epoch 6/50
144/144 [==============================] - 13s 87ms/step - loss: 4.2952e-04 - val_loss: 2.7784e-04
Epoch 7/50
144/144 [==============================] - 13s 91ms/step - loss: 3.8823e-04 - val_loss: 2.7290e-04
Epoch 8/50
144/144 [==============================] - 13s 88ms/step - loss: 3.7688e-04 - val_loss: 2.9363e-04
Epoch 9/50
144/144 [==============================] - 13s 91ms/step - loss: 3.9255e-04 - val_loss: 2.8090e-04
Epoch 10/5

In [24]:

# Eğitilmiş modeli kaydetme
loaded_model.save('model9435.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import load_model

# Veriyi yükleme ve gereksiz sütunları kaldırma
data = pd.read_excel("25-12.xlsx")
data = data.drop(["id", "temp", "w_status"], axis=1)

data = data[(data != 0).all(axis=1)]  # Tüm sütunlar için 0 olmayan satırları seçme
data = data.dropna()  # NaN değerleri içeren satırları kaldırma

# 'date' sütununu datetime formatına çevirme
data['date_v'] = pd.to_datetime(data['date_v'])

# 'date' sütununu Unix zaman damgasına çevirme
data['unix_time'] = data['date_v'].apply(lambda x: int((x - datetime(1970, 1, 1)).total_seconds()))

# Gereksiz sütunu ve orijinal tarih sütununu kaldırma
data = data.drop(["date_v"], axis=1)

# MinMaxScaler ile 'yukh', 'reg' ve 'power' sütunlarını ölçeklendirme
scaler_yukh = MinMaxScaler()
scaler_reg = MinMaxScaler()
scaler_power = MinMaxScaler()

data['yukh_scaled'] = scaler_yukh.fit_transform(data[['yukh']])
data['reg_scaled'] = scaler_reg.fit_transform(data[['reg']])
data['power_scaled'] = scaler_power.fit_transform(data[['power']])

# Ölçeklenmiş sütunları veriden kaldırma
data = data.drop(["yukh", "reg", "power"], axis=1)

# Zaman serisi formatına dönüştürme
def create_time_series_data(data, window_size):
    X, y = [], []

    for i in range(len(data) - window_size):
        # Pencere boyutu kadar veriyi seçme
        window_yukh = data['yukh_scaled'][i:i+window_size].values
        window_reg = data['reg_scaled'][i:i+window_size].values
        window_power = data['power_scaled'][i:i+window_size].values
        target = data['yukh_scaled'][i: i + window_size]

        X.append(np.column_stack((window_yukh, window_reg, window_power)))
        y.append(target)

    return np.array(X), np.array(y)

# Pencere boyutunu belirleme (aynı boyutu kullanmalısınız)
window_size = 64

# Giriş ve hedef verilerini oluşturma
X, y = create_time_series_data(data, window_size)

# Modeli yükleme
model = load_model('model9435.h5')

# Test seti üzerinde tahmin yapma
test_predictions = model.predict(X)

# Test setindeki pencere boyutunda son gerçek değerler
y_true = y[:, -1]

# Test setindeki pencere boyutunda son tahminler
y_pred = test_predictions[:, -1]

# R-kare skoru hesaplama
r2_test = r2_score(y_true, y_pred)

print(f'Modelin Test R-kare skoru: {r2_test}')

1/1 [==============================] - 0s 443ms/step
Modelin Test R-kare skoru: 0.9702710996423141


In [36]:
# Gerçek 'yukh' değerlerini al
yukh_original = scaler_yukh.inverse_transform(y[:, -1].reshape(-1, 1)).flatten()

# Tahmin edilen 'yukh' değerlerini ölçeklendirmeyi tersine çevirme
predicted_yukh_original = scaler_yukh.inverse_transform(test_predictions[:, -1].reshape(-1, 1)).flatten()

# Uzunlukları kontrol et
print(len(yukh_original), len(predicted_yukh_original))

# Veri çerçevesi oluşturma
result_df = pd.DataFrame({'Gerçek Yukh': yukh_original, 'Tahmin Yukh': predicted_yukh_original})

# Sonuçları ekrana yazdırma
print(result_df)

# Veriyi Excel dosyasına yazma
result_df.to_excel("gercek_tahmin_yukh1.xlsx", index=False)


32 32
    Gerçek Yukh  Tahmin Yukh
0        5999.0  5999.634277
1        5999.0  5998.827148
2        5998.0  5997.677246
3        5998.0  5998.845703
4        6000.0  5998.902344
5        5999.0  5999.523926
6        5999.0  6000.011230
7        6001.0  6000.435059
8        6001.0  6000.430664
9        6001.0  6000.421387
10       6001.0  6000.623535
11       5998.0  5998.277832
12       5998.0  5998.020996
13       5998.0  5997.680176
14       5997.0  5996.464355
15       5996.0  5995.219238
16       5994.0  5994.073242
17       5994.0  5993.519043
18       5993.0  5993.145020
19       5992.0  5992.050781
20       5992.0  5992.297852
21       5991.0  5991.727539
22       5990.0  5990.221191
23       5990.0  5990.815918
24       5990.0  5990.538574
25       5988.0  5989.040039
26       5988.0  5988.379883
27       5988.0  5989.701660
28       5989.0  5990.974609
29       5991.0  5992.480957
30       5991.0  5991.477051
31       5991.0  5991.634277
